<a href="https://colab.research.google.com/github/sensiboi/Sales_Funnel_Analysis/blob/main/Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HYPOTHESIS TESTING**

Import Libraries

In [39]:
import pandas as pd
from scipy import stats
from scipy.stats import chi2_contingency

Load Datasets

In [40]:
mql = pd.read_csv("olist_marketing_qualified_leads_dataset.csv")
deals = pd.read_csv("olist_closed_deals_dataset.csv")

Merge Datasets

In [41]:
df = pd.merge(mql, deals, on="mql_id", how="left")

Create binary conversion column

In [42]:
df["converted"] = df["seller_id"].notna().astype(int)

Clean numeric columns

In [43]:
df["declared_product_catalog_size"] = pd.to_numeric(df["declared_product_catalog_size"], errors="coerce")
df["declared_monthly_revenue"] = pd.to_numeric(df["declared_monthly_revenue"], errors="coerce")

Drop missing numeric rows for T test

In [44]:
df_clean = df.dropna(subset=["declared_product_catalog_size", "declared_monthly_revenue"])
df_clean = df_clean[df_clean['declared_product_catalog_size'] > 0]
df_clean = df_clean[df_clean['declared_monthly_revenue'] > 0]

In [45]:
print(df_clean[['declared_product_catalog_size', 'declared_monthly_revenue']].describe())
print(df_clean.dtypes)
print(df_clean.isna().sum())

       declared_product_catalog_size  declared_monthly_revenue
count                      45.000000              4.500000e+01
mean                      227.488889              1.372978e+06
std                       357.144395              7.507545e+06
min                         2.000000              6.000000e+00
25%                        40.000000              1.500000e+04
50%                        75.000000              5.000000e+04
75%                       300.000000              1.300000e+05
max                      2000.000000              5.000000e+07
mql_id                            object
first_contact_date                object
landing_page_id                   object
origin                            object
seller_id                         object
sdr_id                            object
sr_id                             object
won_date                          object
business_segment                  object
lead_type                         object
lead_behaviour_profile 

In [46]:
print(df_clean[['declared_product_catalog_size', 'declared_monthly_revenue']].info())
print(df_clean[['declared_product_catalog_size', 'declared_monthly_revenue']].head(10))
print(df_clean[['declared_product_catalog_size', 'declared_monthly_revenue']].describe())

<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 215 to 7983
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   declared_product_catalog_size  45 non-null     float64
 1   declared_monthly_revenue       45 non-null     float64
dtypes: float64(2)
memory usage: 1.1 KB
None
      declared_product_catalog_size  declared_monthly_revenue
215                          2000.0                  100000.0
420                            15.0                   20000.0
508                           120.0                    6000.0
681                          1000.0                  180000.0
820                            50.0                   30000.0
912                             4.0                       6.0
1124                          400.0                  100000.0
1178                          800.0                  100000.0
1281                          100.0                  150

In [47]:
df_clean['declared_product_catalog_size'] = df_clean['declared_product_catalog_size'].replace('[^0-9.]', '', regex=True)
df_clean['declared_monthly_revenue'] = df_clean['declared_monthly_revenue'].replace('[^0-9.]', '', regex=True)

# Convert to numeric
df_clean['declared_product_catalog_size'] = pd.to_numeric(deals['declared_product_catalog_size'], errors='coerce')
df_clean['declared_monthly_revenue'] = pd.to_numeric(deals['declared_monthly_revenue'], errors='coerce')

# Drop rows with missing or zero values
df_clean = df_clean.dropna(subset=['declared_product_catalog_size', 'declared_monthly_revenue'])
df_clean = df_clean[(df_clean['declared_product_catalog_size'] > 0) & (df_clean['declared_monthly_revenue'] > 0)]

print(df_clean[['declared_product_catalog_size', 'declared_monthly_revenue']].describe())

       declared_product_catalog_size  declared_monthly_revenue
count                            1.0                       1.0
mean                           100.0                  500000.0
std                              NaN                       NaN
min                            100.0                  500000.0
25%                            100.0                  500000.0
50%                            100.0                  500000.0
75%                            100.0                  500000.0
max                            100.0                  500000.0


Seperate Converted vs Non Converted

In [48]:
converted = deals[deals['has_company'] == 1]
not_converted = deals[deals['has_company'] == 0]

Hypothesis 1: Catalog Size vs Conversion (Independent t-test)

In [49]:
t_stat1, p_val1 = stats.ttest_ind(converted['declared_product_catalog_size'],
                                  not_converted['declared_product_catalog_size'],
                                  equal_var=False, nan_policy='omit')

Hypothesis 2: Declared Monthly Revenue vs Conversion (Independent t-test)

In [50]:
t_stat2, p_val2 = stats.ttest_ind(converted['declared_monthly_revenue'],
                                  not_converted['declared_monthly_revenue'],
                                  equal_var=False, nan_policy='omit')

Hypothesis 3: Lead Source vs Conversion (Chi-square test)

In [51]:
df_clean['lead_type'] = df_clean['lead_type'].astype(str)
contingency_table = pd.crosstab(df_clean['lead_type'], df_clean['has_company'])
if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:
  chi2, p_val3, dof, exp = stats.chi2_contingency(contingency_table)

Hypothesis Testing

In [52]:
print("\n--- Hypothesis Testing Results ---")
print(f"H1: Catalog Size vs Conversion → t-stat = {t_stat1:.3f}, p-value = {p_val1:.4f}")
print(f"H2: Monthly Revenue vs Conversion → t-stat = {t_stat2:.3f}, p-value = {p_val2:.4f}")
print(f"H3: Lead Source vs Conversion → Chi² = {chi2:.3f}, p-value = {p_val3:.4f}")


--- Hypothesis Testing Results ---
H1: Catalog Size vs Conversion → t-stat = 2.279, p-value = 0.0319
H2: Monthly Revenue vs Conversion → t-stat = 1.252, p-value = 0.2157
H3: Lead Source vs Conversion → Chi² = 14.483, p-value = 0.0247


Interpretation Table

In [53]:
alpha = 0.05
summary = pd.DataFrame({
    'Hypothesis': [
        'H1: Catalog Size vs Conversion',
        'H2: Monthly Revenue vs Conversion',
        'H3: Lead Source vs Conversion'
    ],
    'Test Used': ['t-test', 't-test', 'Chi-square'],
    'Test Statistic': [t_stat1, t_stat2, chi2],
    'p-value': [p_val1, p_val2, p_val3],
    'Decision': [
        'Reject H₀' if p_val1 < alpha else 'Fail to Reject H₀',
        'Reject H₀' if p_val2 < alpha else 'Fail to Reject H₀',
        'Reject H₀' if p_val3 < alpha else 'Fail to Reject H₀'
    ]
})

print("\n--- Hypothesis Testing Summary ---")
print(summary.to_string(index=False))


--- Hypothesis Testing Summary ---
                       Hypothesis  Test Used  Test Statistic  p-value          Decision
   H1: Catalog Size vs Conversion     t-test        2.279405 0.031908         Reject H₀
H2: Monthly Revenue vs Conversion     t-test        1.252022 0.215674 Fail to Reject H₀
    H3: Lead Source vs Conversion Chi-square       14.482759 0.024685         Reject H₀
